In [10]:
import tkinter as tk
import math
import OpenOPC
import pywintypes
import time
import threading
pywintypes.datetime = pywintypes.TimeType

In [54]:
import tkinter as tk
import math
import OpenOPC

class VisionSystem(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self,  *args, **kwargs)  
        self.parent = None
        self.geometry('1500x1500')
        self.opc = OpenOPC.client()
        self.opc.connect('Matrikon.OPC.Simulation.1')
        self.opc.write(('output.qDone', False))
        self.opc.write(('output.qCmdActive', False))
        self.opc.write(('output.qError', False))
        self.opc.write(('output.qHeartBeatFault', False))
        self.opc.write(('output.qOfflineFault', False))
        self.opc.write(('output.qPartTypeMismatch', False))
        self.buttonState = 0 
        self.startButtonState = 0
        self.iResetFault = tk.IntVar()
        self.reset = 0
        self.partIDValue = tk.StringVar()
        self.partTypeValue = tk.StringVar()
        self.observedIDValue = tk.StringVar()
        self.observedTypeValue = tk.StringVar()
        self.delayValue = tk.StringVar()
        
        
        self.displayFrame = tk.Frame(self)
        self.displayFrame['height'] = 600
        self.displayFrame['width'] = 300
        self.displayFrame.pack()
        self.displayFrame.place(x=1000, y=100)
        self.displayFrame['bg'] = 'yellow'
        self.canvas = tk.Canvas(self.displayFrame)
        self.LEDs = dict()
        self.LEDs['qDone'] = (150, 10, 220, 80)
        self.LEDs['qReady'] = (150, 120, 220, 190)
        self.LEDs['qCmdActive'] = (150, 240, 220, 310) 
        self.LEDs['qPartTypeMismatch'] = (150, 360, 220, 430)
        self.statusPosition = (180, 500)
        self.canvas.pack()
        self.canvas['width'] = 300
        self.canvas['height'] = 600
        self.canvas.create_oval(self.LEDs['qDone'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(184, 90, font="Times 12 bold", text="qDone")
        self.canvas.create_oval(self.LEDs['qReady'], outline="#000000", fill="#81eb8d", width=2)
        self.canvas.create_text(183, 200, font="Times 12 bold", text="qReady")
        self.canvas.create_oval(self.LEDs['qCmdActive'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(181, 320, font="Times 12 bold", text="qCmdActive")
        self.canvas.create_oval(self.LEDs['qPartTypeMismatch'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(180, 440, font="Times 12 bold", text="qPartTypeMismatchFault")
        
        # iTriggerCam Button
        self.iTriggerCamButton = tk.Button(self, text='START INSPECTION', fg='blue', bg='red', command=self.startButtonPressed, height=4, width=20)
        self.iTriggerCamButton.pack()
        self.iTriggerCamButton.place(x=120, y=200)
        
        # iResetCycle Button
        self.iResetCycleButton = tk.Button(self, text='RESET FAULT', fg='blue', bg='red', command=self.resetButtonPressed, height=4, width=20)
        self.iResetCycleButton.pack()
        self.iResetCycleButton.place(x=320, y=200)
        
        
        # iResetFault Button
        self.iResetFaultButton = tk.Checkbutton(self, text='Reset Fault Acknowledgement', padx=20, variable=self.iResetFault, onvalue=1, offvalue=0)
        self.iResetFaultButton.place(x=120, y = 350)
        self.iResetFaultButton.config(font=("Times", 12, 'bold'))
        
        # iTriggerValue
        self.iTriggerValueLabel = tk.Label(self, text="Time Delay", font=("Times", 12, 'bold'))
        self.iTriggerValueLabel.pack()
        self.iTriggerValueLabel.place(x=120, y=450)
        self.iTriggerValue = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.delayValue)
        self.delayValue.set('1000')
        self.iTriggerValue.pack()
        self.iTriggerValue.place(x=270, y=450)
        
        # iPartType
        self.iPartTypeLabel = tk.Label(self, text="Part Type Value", font=("Times", 12, 'bold'))
        self.iPartTypeLabel.pack()
        self.iPartTypeLabel.place(x=420, y=450)
        self.iPartType = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.partTypeValue)
        self.iPartType.pack()
        self.iPartType.place(x=570, y=450)

        # iPartID
        self.iPartIDLabel = tk.Label(self, text="Part ID Value", font=("Times", 12, 'bold'))
        self.iPartIDLabel.pack()
        self.iPartIDLabel.place(x=720, y=450)
        self.iPartID = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.partIDValue)
        self.iPartID.pack()
        self.iPartID.place(x=870, y=450)
        
        # iObservedType
        self.iObservedTypeLabel = tk.Label(self, text="Observed Type Value", font=("Times", 12, 'bold'))
        self.iObservedTypeLabel.pack()
        self.iObservedTypeLabel.place(x=420, y=500)
        self.iObservedType = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.observedTypeValue)
        self.iObservedType.pack()
        self.iObservedType.place(x=570, y=500)

        # iObservedID
        self.iObservedIDLabel = tk.Label(self, text="Observed ID Value", font=("Times", 12, 'bold'))
        self.iObservedIDLabel.pack()
        self.iObservedIDLabel.place(x=720, y=500)
        self.iObservedID = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.observedIDValue)
        self.iObservedID.pack()
        self.iObservedID.place(x=870, y=500)

    def setLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#81eb8d", width=2)
    def resetLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#f54e42", width=2)
    def cycleTerminationProcedure(self):
        self.opc.write(('output.qReady', True))
        self.setLED('qReady')
        self.startButtonState = 0
        self.iTriggerCamButton['bg'] = 'red'
        self.iTriggerCamButton['text']  = 'START INSPECTION'
        self.opc.write(('output.qDone', False))
        self.resetLED('qDone')
        self.opc.write(('output.qCmdActive', False))
        self.resetLED('qCmdActive')
        self.opc.write(('output.qPartTypeMismatch', False))
        self.resetLED('qPartTypeMismatch')
        
    def compare(self):
        if self.reset == 0 and self.startButtonState == 1:
            self.opc.write(('output.qReady', False))
            self.resetLED('qReady')
            self.opc.write(('output.qCmdActive', True))
            self.setLED('qCmdActive')
            if self.partTypeValue.get() == self.observedTypeValue.get() and self.partIDValue.get() == self.observedIDValue.get():
                self.opc.write(('output.qDone', True))
                self.setLED('qDone')
                self.opc.write(('output.qPartTypeMismatch', False))       
            else:
                self.opc.write(('output.qDone', False))
                self.resetLED('qDone')
                self.opc.write(('output.qPartTypeMismatch', True))
                self.setLED('qPartTypeMismatch')
            self.reset = 1
            self.after(int(self.delayValue.get()), lambda : self.cycleTerminationProcedure()) 
            
            
        self.after(100, lambda : self.compare()) 
    def resetButtonPressed(self):
        self.reset = 1        
        self.opc.write(('output.qCmdActive', False)) 
        self.resetLED('qCmdActive')
        self.opc.write(('output.qDone', False))
        self.resetLED('qDone')
        self.opc.write(('output.qPartTypeMismatch', False))
        self.resetLED('qPartTypeMismatch')
    def startButtonPressed(self):
        if self.startButtonState == 0:
            self.opc.write(('output.qReady', True))
            self.setLED('qReady')
            self.reset = 0
            self.startButtonState = 1
            self.iTriggerCamButton['bg'] = 'green'
            self.iTriggerCamButton['text']  = 'STOP INSPECTION'
            self.opc.write(('output.qCmdActive', True))
            self.setLED('qCmdActive')
        else:
            self.opc.write(('output.qReady', False))
            self.resetLED('qReady')
            self.startButtonState = 0
            self.iTriggerCamButton['bg'] = 'red'
            self.iTriggerCamButton['text']  = 'START INSPECTION'

    def run(self):
        self.after(100,lambda : self.compare())  
        self.mainloop()




In [56]:
V = VisionSystem()
V.run()